<a href="https://colab.research.google.com/github/peculab/PythonAI4Beginners/blob/main/%E7%AC%AC%E5%8D%81%E5%85%AD%E9%80%B1_%E5%9F%BA%E7%A4%8E%E8%87%AA%E7%84%B6%E8%AA%9E%E8%A8%80%E8%99%95%E7%90%86_NLP_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

分析 PTT 電影版
https://www.ptt.cc/bbs/movie/index.html

In [12]:
import nbformat
from glob import glob

for path in glob('*.ipynb'):
    nb = nbformat.read(path, as_version=4)
    if 'widgets' in nb['metadata']:
        del nb['metadata']['widgets']
    nbformat.write(nb, path)

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time

In [ ]:
PTT_URL = 'https://www.ptt.cc/bbs/movie/index.html'  # PTT 電影版網址
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'} # 設定 User-Agent，避免被 PTT 擋掉

In [ ]:
def get_articles(url, pages=2): # pages=2 代表爬取最近兩頁
    all_articles = []
    for page in range(pages):
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = []
        for r_ent in soup.find_all(class_="r-ent"):
            try:
                link = r_ent.find('a')['href']
                title = r_ent.find('a').text
                date = r_ent.find(class_='date').text.strip()
                author = r_ent.find(class_='author').text
                link = 'https://www.ptt.cc' + link
                articles.append({'title': title, 'link': link, 'date': date, 'author':author})

            except:
                pass

        next_page = soup.find('a', string='‹ 上頁')
        if next_page:
            url = 'https://www.ptt.cc' + next_page['href']
            time.sleep(2) # 避免過度請求
        else:
            break
        all_articles.extend(articles)
    return all_articles

def get_article_content(link):
  try:
      response = requests.get(link, headers=HEADERS)
      soup = BeautifulSoup(response.text, 'html.parser')

      main_content = soup.find(id='main-content')
      #移除掉標籤跟作者資訊
      all_text = main_content.text.split('--')[0]

      pre_texts = main_content.find_all(class_='article-metaline')
      for pre_text in pre_texts:
        all_text = all_text.replace(pre_text.text,'')
      all_text = all_text.replace('\n','').strip()
      return all_text
  except:
      return "無法讀取內文"

articles = get_articles(PTT_URL)
for article in articles:
  article['content'] = get_article_content(article['link'])
  print("標題:"+article['title'])
  print("連結:"+article['link'])

標題:[新聞]評價炸裂卻不悔！席德妮史威尼演《蜘蛛夫人
連結:https://www.ptt.cc/bbs/movie/M.1748013307.A.34F.html
標題:[新聞]「兔腳」到底是什麼？《不可能的任務最終
連結:https://www.ptt.cc/bbs/movie/M.1748013489.A.D93.html
標題:[普雷] 不可能的任務 最終清算 沒有比較沒有傷害
連結:https://www.ptt.cc/bbs/movie/M.1748014439.A.1F9.html
標題:[負微雷] 尋秘不老泉:蓋瑞奇好色
連結:https://www.ptt.cc/bbs/movie/M.1748016451.A.EC3.html
標題:[普雷] 鬼玩人:復活 - 媽媽請你母湯疼
連結:https://www.ptt.cc/bbs/movie/M.1748029258.A.113.html
標題:[  雷] 不可能任務放：最終清算 真的不可能
連結:https://www.ptt.cc/bbs/movie/M.1748045616.A.A2C.html
標題:[公告] 電影板板規 2022/12/5
連結:https://www.ptt.cc/bbs/movie/M.1630756788.A.1FE.html
標題:[公告] 禁政治版規 及 投票結果
連結:https://www.ptt.cc/bbs/movie/M.1694784041.A.89F.html
標題:Re: [討論] 阿湯哥飾演的伊森·韓特有軍人身份嗎？
連結:https://www.ptt.cc/bbs/movie/M.1747976250.A.101.html
標題:[請益] 如果一部電影
連結:https://www.ptt.cc/bbs/movie/M.1747978728.A.2B9.html
標題:Re: [贈票] 魏斯安德森【腓尼基計劃】搶先看
連結:https://www.ptt.cc/bbs/movie/M.1747982765.A.82A.html
標題:[新聞] 《復仇者5、6》延後上映 傳薩諾斯將回歸客
連結:https://www.ptt.cc/bbs/movie/M.1747985075.A.165.html
標題:[情報] 8號出口預告
連結:http

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1u7BOjJoRt7kfAXBG0d_WPcSjfLwNDlC94JPLjk4KaK8/edit?usp=sharing')
worksheet = spreadsheet.worksheet('test2')

In [ ]:
articles

[{'title': '[新聞]評價炸裂卻不悔！席德妮史威尼演《蜘蛛夫人',
  'link': 'https://www.ptt.cc/bbs/movie/M.1748013307.A.34F.html',
  'date': '5/23',
  'author': 'XDGEE',
  'content': '看板movie評價炸裂卻不悔！席德妮史威尼演《蜘蛛夫人》？她坦白很大原因是為了家人評價炸裂卻不後悔！席德妮史威尼為何要接演《蜘蛛夫人》？她坦白很大原因是為了家人憑藉《高校十八禁》、《白蓮花大飯店》逐漸嶄露頭角的席德妮史威尼，近年來炙手可熱，並在多部影視作品中有令人印象深刻的表現。不過，由索尼影業發行的《蜘蛛夫人》卻成了她演藝事業的「特別作品」，根據她的說法，當初接下這個角色，很大一部分是為了討家人歡心。在接受《帝國雜誌》專訪時，席德妮史威尼回憶起《蜘蛛夫人》的拍攝經歷，儘管這部電影最終遭遇口碑滑鐵盧，她仍對拍攝過程津津樂道，認為這為她帶來寶貴的曝光機會。「參與一部比自己更具知名度的作品總是令人興奮。當時我還沒有過大製片廠電影的經驗，而我剛成立了自己的製片公司Fifty-Fifty Films。我手中有一些希望帶進工作室的項目，也需要提升自己在好萊塢大片市場中的價值和地位。」她透露，參演《蜘蛛夫人》還有另一個溫馨理由——讓年幼的表親們能看到她的表演。「另外，我也想拍一些讓表親們能觀賞的電影。他們很多還是青少年，對我的工作內容一知半解。我覺得能參與一部讓他們覺得酷炫有趣的電影，是件很棒的事。」遺憾的是，《蜘蛛夫人》最終在觀眾和影評人心中都留下慘痛印象，成為她演藝生涯迄今評價最低的作品。不過，席德妮史威尼並未因負評而減損對這部電影的熱忱。「說實話，我度過了愉快的拍攝時光，這對我而言最為重要。我認為如果你享受工作過程，票房表現就不是那麼關鍵了。當然，你仍希望作品能獲得讚賞和成功，因為這意味著所有參與者都獲得了成功。」此前，席德妮史威尼曾多次公開闡述參演《蜘蛛夫人》的動機，坦言這是一個「策略性商業決策」，有助於與索尼高層建立良好關係。此外，這部電影也為她帶來豐厚報酬，讓她在多次訪談中對這部作品表達感謝。《蜘蛛夫人》的演員陣容還包括達珂塔強生、賽麗絲歐康納、伊莎貝拉莫塞及亞當史考特。這部電影全球票房僅達 1 億美元，不僅成為「索尼蜘蛛人宇宙」系列中評價最差的作品，更在

In [ ]:
for article in articles:
    row = [article['title'], article['link'], article['date'], article['author'], article['content']]
    worksheet.append_row(row)

print("資料已成功寫入 Google Sheet！")

資料已成功寫入 Google Sheet！


In [ ]:
# 讀取所有數據
rows = worksheet.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [ ]:
df

,[新聞]評價炸裂卻不悔！席德妮史威尼演《蜘蛛夫人,https://www.ptt.cc/bbs/movie/M.1748013307.A.34F.html,5/23,XDGEE,看板movie評價炸裂卻不悔！席德妮史威尼演《蜘蛛夫人》？她坦白很大原因是為了家人評價炸裂卻不後悔！席德妮史威尼為何要接演《蜘蛛夫人》？她坦白很大原因是為了家人憑藉《高校十八禁》、《白蓮花大飯店》逐漸嶄露頭角的席德妮史威尼，近年來炙手可熱，並在多部影視作品中有令人印象深刻的表現。不過，由索尼影業發行的《蜘蛛夫人》卻成了她演藝事業的「特別作品」，根據她的說法，當初接下這個角色，很大一部分是為了討家人歡心。在接受《帝國雜誌》專訪時，席德妮史威尼回憶起《蜘蛛夫人》的拍攝經歷，儘管這部電影最終遭遇口碑滑鐵盧，她仍對拍攝過程津津樂道，認為這為她帶來寶貴的曝光機會。「參與一部比自己更具知名度的作品總是令人興奮。當時我還沒有過大製片廠電影的經驗，而我剛成立了自己的製片公司Fifty-Fifty Films。我手中有一些希望帶進工作室的項目，也需要提升自己在好萊塢大片市場中的價值和地位。」她透露，參演《蜘蛛夫人》還有另一個溫馨理由——讓年幼的表親們能看到她的表演。「另外，我也想拍一些讓表親們能觀賞的電影。他們很多還是青少年，對我的工作內容一知半解。我覺得能參與一部讓他們覺得酷炫有趣的電影，是件很棒的事。」遺憾的是，《蜘蛛夫人》最終在觀眾和影評人心中都留下慘痛印象，成為她演藝生涯迄今評價最低的作品。不過，席德妮史威尼並未因負評而減損對這部電影的熱忱。「說實話，我度過了愉快的拍攝時光，這對我而言最為重要。我認為如果你享受工作過程，票房表現就不是那麼關鍵了。當然，你仍希望作品能獲得讚賞和成功，因為這意味著所有參與者都獲得了成功。」此前，席德妮史威尼曾多次公開闡述參演《蜘蛛夫人》的動機，坦言這是一個「策略性商業決策」，有助於與索尼高層建立良好關係。此外，這部電影也為她帶來豐厚報酬，讓她在多次訪談中對這部作品表達感謝。《蜘蛛夫人》的演員陣容還包括達珂塔強生、賽麗絲歐康納、伊莎貝拉莫塞及亞當史考特。這部電影全球票房僅達 1 億美元，不僅成為「索尼蜘蛛人宇宙」系列中評價最差的作品，更在金酸莓獎上囊括最差影片、最差女主角和最差劇本三項「殊榮」。儘管如此，該片登陸 Netflix 後仍創下亮眼收視表現。https://news.agentm.tw/357907/
0,[新聞]「兔腳」到底是什麼？《不可能的任務最終,https://www.ptt.cc/bbs/movie/M.1748013489.A.D9...,5/23,XDGEE,看板movie「兔腳」到底是什麼？歷時19年《不可能的任務最終清算》終於揭露系列最大謎團還記...
1,[普雷] 不可能的任務 最終清算 沒有比較沒有傷害,https://www.ptt.cc/bbs/movie/M.1748014439.A.1F...,5/23,nyybos8812,看板movie先說劇情其實蠻制式的 沒想到這系列的最後是ai想要毀滅世界 美國總統當聖人這種...
2,[負微雷] 尋秘不老泉:蓋瑞奇好色,https://www.ptt.cc/bbs/movie/M.1748016451.A.EC...,5/24,arsl400,看板movie《尋秘不老泉》（Fountain of Youth）是一部2025年美國搶劫冒...
3,[普雷] 鬼玩人:復活 - 媽媽請你母湯疼,https://www.ptt.cc/bbs/movie/M.1748029258.A.11...,5/24,LittleDiDi,看板movie大家好我是專看恐怖片的LittleDiDi將近一年沒發文了就是來跟大家打招呼的...
4,[ 雷] 不可能任務放：最終清算 真的不可能,https://www.ptt.cc/bbs/movie/M.1748045616.A.A2...,5/24,jack5u06d93,看板movie比起上一集跳傘這集真的刺激很多很多地方片段都很刺激我是看日日新至尊音效真的很讚...
5,[公告] 電影板板規 2022/12/5,https://www.ptt.cc/bbs/movie/M.1630756788.A.1F...,9/04,ckshchen,看板movie※ [本文轉錄自 ckshchen 信箱]作者: TtTt4 (夜間飛行) 看...
6,[公告] 禁政治版規 及 投票結果,https://www.ptt.cc/bbs/movie/M.1694784041.A.89...,9/15,ckshchen,看板movie政治相關版規(新版/寬鬆版):註:下述提到的政治都是非講到台灣現況的政治電影延...
7,Re: [討論] 阿湯哥飾演的伊森·韓特有軍人身份嗎？,https://www.ptt.cc/bbs/movie/M.1747976250.A.10...,5/23,sonyxeperiau,看板movie※ 引述《sonyxeperiau (son)》之銘言：: 防雷線……………:...
8,[請益] 如果一部電影,https://www.ptt.cc/bbs/movie/M.1747978728.A.2B...,5/23,qq326,看板movie串流都沒有 影音出租店也沒有(台中剩不到5間錦城 我問了3間都沒有)youtu...
9,Re: [贈票] 魏斯安德森【腓尼基計劃】搶先看,https://www.ptt.cc/bbs/movie/M.1747982765.A.82...,5/23,cecilia1220,"看板movie恭喜以下得獎者，請於5月26日上午10:00前主動私訊回覆得獎者資料,恕不接受..."


文本情感分析 (Sentiment Analysis)：

    目的： 判斷影評、討論區留言等文本的情感傾向，例如正面、負面或中立。
    應用場景： 了解觀眾對電影的喜好程度、分析電影口碑、監測社群媒體上的討論趨勢。

In [ ]:
!pip install transformers
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="uer/roberta-base-finetuned-dianping-chinese")
result = classifier("這部電影非常好看！")
print(result)

result = classifier("這部電影真是爛透了！")
print(result)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'positive (stars 4 and 5)', 'score': 0.9658221006393433}]
[{'label': 'negative (stars 1, 2 and 3)', 'score': 0.9962218999862671}]


In [ ]:
result[0]['score']

0.9962218999862671

In [ ]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [ ]:
import google.generativeai as genai
import pandas as pd
import json

genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

def analyze_social_media_sentiment(row):
    # 將 DataFrame 轉換為適合 prompt 的字串格式
    prompt = f"""
    請分析以下社交媒體上對特定事件的討論，判斷整體風向（正面、負面、中性），並簡要說明你的判斷依據。
    請以 JSON 格式回覆，包含 'sentiment'（整體風向）和 'reasoning'（判斷依據）兩個欄位。

    討論內容：
    {row}
    """

    try:
        response = model.generate_content(prompt)
        response_text = response.text
    except Exception as e:
        print(f"發生錯誤：{e}")

    return response_text # 確保一定會回傳值

In [ ]:
df['內容']

,內容
0,看板movie剛剛看完了小時候老師常常放的影片三個傻瓜藍丘許多人看完都認為他是勇於衝撞體制 ...
1,看板moviehttps://mdpr.jp/cinema/detail/4448144ht...
2,看板movie 日本電影《雪風YUKIKAZE》2025年8月上映 無意間在Yout...
3,看板movie我蠻驚訝的是，迪士尼竟然沒有著手發展以母獅群為主角的衍生作品。一是女性主角是迪...
4,看板movie※ [本文轉錄自 ckshchen 信箱]作者: TtTt4 (夜間飛行) 看...
5,看板movie政治相關版規(新版/寬鬆版):註:下述提到的政治都是非講到台灣現況的政治電影延...
6,看板movie事實上孩之寶在變形金剛的IP上面並沒有賠一屁股，變形金剛：萬獸崛起的票房達到成...
7,看板movie防雷我還滿喜歡這個導演作品中的生活哲學那淡淡的風格讓人沉浸在劇情中喜歡的會很舒...
8,看板movie新聞網址：https://reurl.cc/zpAzkpUDN / 資深演員林...
9,看板movie看到推文有提到G1造型不符合現代審美我以前也這麼覺得直到大黃蜂上映，這部的人物...


In [ ]:
df['sentiment'] = df['內容'].apply(lambda x: analyze_social_media_sentiment(x))

In [ ]:
df['sentiment']

,sentiment
0,"```json\n{\n ""sentiment"": ""正面"",\n ""reasoning..."
1,"```json\n{\n ""sentiment"": ""正面"",\n ""reasoning..."
2,"```json\n{\n ""sentiment"": ""中性"",\n ""reasoning..."
3,"```json\n{\n ""sentiment"": ""正面"",\n ""reasoning..."
4,"```json\n{\n ""sentiment"": ""Neutral"",\n ""reas..."
5,"```json\n{\n ""sentiment"": ""中性"",\n ""reasoning..."
6,"```json\n{\n ""sentiment"": ""Mixed"",\n ""reason..."
7,"```json\n{\n ""sentiment"": ""正面"",\n ""reasoning..."
8,"```json\n{\n ""sentiment"": ""負面"",\n ""reasoning..."
9,"```json\n{\n ""sentiment"": ""Mixed"",\n ""reason..."


In [ ]:
json_string = df['sentiment'][1]
json_string

'```json\n{\n  "sentiment": "正面",\n  "reasoning": "新聞稿整體呈現正面積極的語氣，主要基於以下幾個方面：\\n\\n1. **作品人氣:**  漫畫原作銷量突破90萬冊，說明作品本身擁有相當高的人氣基礎，為電影改編奠定了良好的觀眾基礎。\\n2. **演員陣容:**  選用的演員福本莉子和八木勇征都是當下受矚目的演員，他們的參與提升了電影的期待值。演員們對於角色和電影的積極評價也增加了正面氛圍。\\n3. **製作團隊:**  導演松本花奈曾執導多部成功的青春題材電影，經驗豐富，也增加了電影質量的保證。\\n4. **原作者及製作方態度:** 原作者和製作方都對電影改編表示高度期待和信心，並肯定了劇本和演員陣容。\\n5. **劇情簡介:**  “純真愛情故事”、“經典愛情故事”等詞語營造了溫馨浪漫的氛圍，更容易引起正面共鳴。"\n}\n```\n'

In [ ]:
cleaned_json_string = json_string.replace("\\n", "").replace("\n", "").replace("```json", '').replace("```", '')

In [ ]:
cleaned_json_string

'{  "sentiment": "正面",  "reasoning": "新聞稿整體呈現正面積極的語氣，主要基於以下幾個方面：1. **作品人氣:**  漫畫原作銷量突破90萬冊，說明作品本身擁有相當高的人氣基礎，為電影改編奠定了良好的觀眾基礎。2. **演員陣容:**  選用的演員福本莉子和八木勇征都是當下受矚目的演員，他們的參與提升了電影的期待值。演員們對於角色和電影的積極評價也增加了正面氛圍。3. **製作團隊:**  導演松本花奈曾執導多部成功的青春題材電影，經驗豐富，也增加了電影質量的保證。4. **原作者及製作方態度:** 原作者和製作方都對電影改編表示高度期待和信心，並肯定了劇本和演員陣容。5. **劇情簡介:**  “純真愛情故事”、“經典愛情故事”等詞語營造了溫馨浪漫的氛圍，更容易引起正面共鳴。"}'

In [ ]:
def get_sentiment_and_reasoning(json_string):
    cleaned_json_string = json_string.replace("\\n", "").replace("\n", "").replace("```json", '').replace("```", '')
    try:
        data = json.loads(cleaned_json_string)
        sentiment = data["sentiment"]
        reasoning = data["reasoning"]
        print(f"情感：{sentiment}")
        print(f"理由：{reasoning}")
    except json.JSONDecodeError as e:
        print(f"JSON 解析錯誤：{e}")
        print(f"原始字串：{json_string}")
        print(f"清理後字串：{cleaned_json_string}")
    return sentiment, reasoning # 確保一定會回傳值

In [ ]:
df['sentiment_json'] = df['sentiment'].apply(lambda x: get_sentiment_and_reasoning(x))

情感：正面
理由：這段文字主要表達了對電影《三個傻瓜》和《春風化雨》的正面評價。文中肯定了主角藍丘的熱情、堅持和為朋友付出的精神，並將其與《春風化雨》中的基汀老師聯繫起來，讚揚他們勇於追求自我、不墨守成規的精神。  作者認為電影的主旨並非單純的衝撞體制，而是鼓勵人們追隨自己的興趣，勇敢選擇自己的人生道路，並對社會的價值觀念提出了質疑，認為不應將選擇專業局限於特定領域。整體語氣積極向上，充滿對理想人生的嚮往和對電影主題的深刻理解，因此判斷整體風向為正面。
情感：正面
理由：新聞稿整體呈現正面積極的語氣，主要基於以下幾個方面：1. **作品人氣:**  漫畫原作銷量突破90萬冊，說明作品本身擁有相當高的人氣基礎，為電影改編奠定了良好的觀眾基礎。2. **演員陣容:**  選用的演員福本莉子和八木勇征都是當下受矚目的演員，他們的參與提升了電影的期待值。演員們對於角色和電影的積極評價也增加了正面氛圍。3. **製作團隊:**  導演松本花奈曾執導多部成功的青春題材電影，經驗豐富，也增加了電影質量的保證。4. **原作者及製作方態度:** 原作者和製作方都對電影改編表示高度期待和信心，並肯定了劇本和演員陣容。5. **劇情簡介:**  “純真愛情故事”、“經典愛情故事”等詞語營造了溫馨浪漫的氛圍，更容易引起正面共鳴。
情感：中性
理由：討論主要圍繞電影《雪風YUKIKAZE》的上映消息、劇情背景（二戰日本軍艦）以及台灣與日本歷史關聯（雪風艦在台服役經歷）。  帖文本身並未表達明顯的正面或負面情緒，只是陳述事實並提出關於台灣片商是否引進電影的疑問。  雖然涉及二戰歷史敏感議題，但討論內容並未展現對該議題的負面或批判性情緒，也沒有明顯的讚揚或支持。因此，整體風向判斷為中性。
情感：正面
理由：該討論表達了對迪士尼缺乏以母獅群為主角的衍生作品的遺憾和期待。文中多次使用正面詞彙，例如「帥呆了」、「帥」、「存活關鍵」，並肯定了母獅在電影中的角色和能力，展現出對母獅角色的喜愛和對相關衍生作品的強烈期待。整體語氣積極，表達了對此類作品的渴望，而非批評或負面評價。
情感：Neutral
理由：This text is a set of rules for a movie discussion board.  It outlines acceptable and unacceptable be

In [ ]:
df['sentiment_json']

,sentiment_json
0,"(正面, 這段文字主要表達了對電影《三個傻瓜》和《春風化雨》的正面評價。文中肯定了主角藍丘的..."
1,"(正面, 新聞稿整體呈現正面積極的語氣，主要基於以下幾個方面：1. **作品人氣:** 漫..."
2,"(中性, 討論主要圍繞電影《雪風YUKIKAZE》的上映消息、劇情背景（二戰日本軍艦）以及台..."
3,"(正面, 該討論表達了對迪士尼缺乏以母獅群為主角的衍生作品的遺憾和期待。文中多次使用正面詞彙..."
4,"(Neutral, This text is a set of rules for a mo..."
5,"(中性, 討論內容主要圍繞PTT movie版關於政治討論的新版規的投票結果和規則細節，包含..."
6,"(Mixed, The discussion presents a mixed sentim..."
7,"(正面, 評論者整體表達了對電影的喜愛。例如，他稱讚了導演的生活哲學、淡淡的風格、選角、題材..."
8,"(負面, 雖然新聞報導林照雄先生的過世，並提及女兒林姿佑的悼念之情，整體基調是哀傷和惋惜。網..."
9,"(Mixed, The discussion shows a mixed sentiment..."
